# Continuous Control

---
### 1. Start the Environment

We begin by importing the necessary packages.

In [3]:
!pip -q install ./python

In [4]:
from unityagents import UnityEnvironment
import numpy as np
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
import time
%matplotlib inline

## Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [5]:
env = UnityEnvironment(file_name='./Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [6]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Trainning 



In [8]:
def ddpg(n_episodes=2000, max_t=700,reset_time=100):
    start_time = time.time()
    scores_deque = deque(maxlen=100)
    scores_mean_agent = []
    scores_mean = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] 
        states = env_info.vector_observations
        agent.reset()
        scores = np.zeros(num_agents)
        for t in range(max_t):
            if t % reset_time == 0:
                agent.reset()
            actions = agent.act(states,add_noise=True)
            #next_state, reward, done, _ = env.step(action)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations  # get the next state
            rewards = env_info.rewards  # get the reward
            dones = env_info.local_done

            agent.step(states, actions, rewards, next_states, dones, t)
          
            
            states = next_states
            scores += rewards
            if np.any(dones):
                break       
        score = np.mean(scores)
        scores_deque.append(score)
        scores_mean_agent.append(score)
        scores_mean.append(np.mean(scores_deque))
        
       
        
        if i_episode % 10 == 0 :
            duration = (int)(time.time() - start_time)  
            print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}\tMin: {:.2f}\tMax: {:.2f}\tepsilon {:.4f}\tTime: {:02}:{:02}:{:02}'\
                  .format(i_episode, np.mean(scores_deque),score,  np.min(scores), np.max(scores),agent.epsilon,\
                          duration//3600, duration%3600//60, duration%60))
        else:
             print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}\tMin: {:.2f} \tMax: {:.2f}\tepsilon {:.2f}'\
              .format(i_episode, np.mean(scores_deque), score, np.min(scores), np.max(scores),agent.epsilon), end="")
                
        if np.mean(scores_deque) >= 30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode - print_every, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), save_or_load_path)
            torch.save(agent.critic_local.state_dict(), save_or_load_path)
            break  
    return scores_mean_agent,scores_mean



In [ ]:

from workspace_utils import active_session 
from ddpg_agent import Agent


env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
action_size = brain.vector_action_space_size           # get the action size
state_size = states.shape[1]                           # get the state size
random_seed = 2
agent = Agent(state_size,action_size,random_seed)      # init the agent

with active_session():
    scores,scores_mean= ddpg()

    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(1, len(scores)+1), scores,label="score")
    plt.plot(np.arange(1, len(scores_mean)+1), scores_mean,label="avg score")
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

cpu
Episode 10	Average Score: 0.26	Score: 0.22	Min: 0.00	Max: 0.53	epsilon 0.9300	Time: 00:03:27
Episode 20	Average Score: 0.48	Score: 0.87	Min: 0.00	Max: 2.55	epsilon 0.8600	Time: 00:07:05
Episode 30	Average Score: 0.97	Score: 2.78	Min: 0.58	Max: 5.27	epsilon 0.7900	Time: 00:10:47
Episode 40	Average Score: 1.69	Score: 5.29	Min: 2.63	Max: 7.71	epsilon 0.7200	Time: 00:14:28
Episode 50	Average Score: 2.63	Score: 6.28	Min: 2.07	Max: 15.36	epsilon 0.6500	Time: 00:18:07
Episode 60	Average Score: 3.67	Score: 10.29	Min: 5.81	Max: 15.91	epsilon 0.5800	Time: 00:21:44
Episode 70	Average Score: 5.00	Score: 16.52	Min: 9.32	Max: 22.82	epsilon 0.5100	Time: 00:25:23
Episode 80	Average Score: 6.35	Score: 17.93	Min: 7.67	Max: 23.55	epsilon 0.4400	Time: 00:29:04
Episode 90	Average Score: 7.55	Score: 14.52	Min: 3.87	Max: 22.23	epsilon 0.3700	Time: 00:33:01
Episode 100	Average Score: 8.61	Score: 19.24	Min: 15.38	Max: 24.97	epsilon 0.3000	Time: 00:37:02
Episode 110	Average Score: 10.70	Score: 22.34	Min: 16

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```